In [2]:
#Setting up paths for data
data_root = "/home/jovyan/nnunet2-mig-7g-80gb-datavol-1/data/Brain-Tumor-Classification" 

tr_base  = data_root+"/Training"
te_base  = data_root+"/Testing"

In [3]:
#Setting up Transforms 
from pathlib import Path
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler # remember subset when you divide into training and validation
from torchvision import datasets, transforms, utils, models # add models when you import pretrained models for transfer learning
import os
import torch
import numpy as np

IMG_SIZE = 512

def zscore(x):  # MRI-friendly
    return (x - x.mean()) / (x.std() + 1e-8)

train_tf = transforms.Compose([
    transforms.Lambda(lambda im: im.convert('L')),                 # use 'RGB' if truly 3-ch
    transforms.Grayscale(num_output_channels=3), 
    transforms.Resize((IMG_SIZE, IMG_SIZE), antialias=True),
    transforms.RandomAffine(degrees=10, translate=(0.02,0.02), scale=(0.95,1.05)),
    transforms.RandomApply([transforms.GaussianBlur(3, (0.1, 1.0))], p=0.2),
    transforms.ColorJitter(brightness=0.10, contrast=0.10),
    transforms.ToTensor(),
    #transforms.Lambda(zscore),
])

eval_tf = transforms.Compose([
    transforms.Lambda(lambda im: im.convert('L')),
    transforms.Grayscale(num_output_channels=3), 
    transforms.Resize((IMG_SIZE, IMG_SIZE), antialias=True),
    transforms.ToTensor(),
    #transforms.Lambda(zscore),
])

In [4]:
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

def get_device():
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def to_device(batch, device):
    x, y = batch
    return x.to(device, non_blocking=True), y.to(device, non_blocking=True)

In [5]:
trainset=datasets.ImageFolder(tr_base, transform=train_tf)
valset = datasets.ImageFolder(tr_base, transform=eval_tf)
testset=datasets.ImageFolder(te_base, transform=eval_tf)


print(testset.classes)
print(trainset.classes)
print(valset.classes)

val_fraction=0.2

idxs = list(range(len(valset)))
rng = np.random.default_rng(42)
rng.shuffle(idxs)

n_val = int(len(idxs) * val_fraction)
val_idx = idxs[:n_val]
train_idx = idxs[n_val:]

trainset = Subset(trainset, train_idx)
valset   = Subset(valset,   val_idx)

class_names = datasets.ImageFolder(tr_base).classes

['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']


In [6]:
def get_targets_from_dataset(ds) -> np.ndarray:
    """Return class indices for ImageFolder or Subset(ImageFolder)."""
    if isinstance(ds, Subset):
        base, idxs = ds.dataset, ds.indices
        if hasattr(base, "targets"):
            return np.asarray(base.targets, dtype=int)[idxs]
        return np.asarray([base.samples[i][1] for i in idxs], dtype=int)
    # ImageFolder
    if hasattr(ds, "targets"):
        return np.asarray(ds.targets, dtype=int)
    return np.asarray([y for _, y in ds.samples], dtype=int)

In [7]:

# assume you already created trainset (ImageFolder or Subset(ImageFolder))
targets = get_targets_from_dataset(trainset)              # shape [N]
num_classes = int(targets.max() + 1)

# inverse-frequency class weights (simple & effective)
class_counts  = np.bincount(targets, minlength=num_classes).astype(float)
class_weights = 1.0 / np.maximum(class_counts, 1.0)      # avoid div/0
sample_weights = class_weights[targets]                   # per-sample weight

sampler = WeightedRandomSampler(
    weights=torch.as_tensor(sample_weights, dtype=torch.double),
    num_samples=len(sample_weights),   # one epoch ~= dataset size
    replacement=True                   # allows oversampling of minority class
)

BATCH = 32
WORKERS = 4


train_loader = DataLoader(
    trainset,
    batch_size=BATCH,
    shuffle=False,
    sampler=sampler,
    num_workers=WORKERS,
    pin_memory=True,
    drop_last=True,                    # optional: keeps batch size stable
)

# validation/test loaders stay standard (no sampler)
val_loader = DataLoader(valset, batch_size=64, shuffle=False, num_workers=WORKERS, pin_memory=True)

In [8]:
import torch.nn as nn
from torchvision import models

def build_model(n_classes, freeze_backbone=False):
    """
    Build a ResNeXt-50 (32x4d) model pretrained on ImageNet, 
    replace classifier for custom number of classes,
    and optionally freeze backbone layers.
    """
    # 1️⃣ Load pretrained ResNeXt-50
    model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.IMAGENET1K_V1)

    # 2️⃣ Replace the final fully connected layer
    in_feats = model.fc.in_features
    model.fc = nn.Linear(in_feats, n_classes)

    # 3️⃣ Optionally freeze feature extractor
    if freeze_backbone:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.fc.parameters():
            param.requires_grad = True

    return model

model = build_model(n_classes=4, freeze_backbone=False)
print(model)

# Verify output
import torch
x = torch.randn(2, 3, 512, 512)
out = model(x)
print(out.shape)  # → [2, 4]

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /home/jovyan/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 326MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [25]:
# ----------------------------
# 3) Training utilities
# ----------------------------
class EarlyStopper:
    def __init__(self, patience=8, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.best = None
        self.count = 0

    def step(self, metric):
        if self.best is None or metric > self.best + self.min_delta:
            self.best = metric
            self.count = 0
            return False  # do not stop

In [26]:
import contextlib
import torch
import torch.nn.functional as F

def epoch_run(model, loader, criterion, device, train: bool, use_amp: bool,
              optimizer=None, scaler=None):
    assert (train and optimizer is not None and scaler is not None) or (not train)

    model.train() if train else model.eval()

    total = 0
    running_loss = 0.0
    correct = 0

    # no graph during evaluation
    cm = contextlib.nullcontext() if train else torch.no_grad()

    with cm:
        for xb, yb in loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)

            with torch.amp.autocast('cuda', enabled=(use_amp and device.type == 'cuda')):
                logits = model(xb)
                loss = criterion(logits, yb)

            if train:
                optimizer.zero_grad(set_to_none=True)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

            running_loss += loss.detach().float().item() * xb.size(0)
            pred = logits.argmax(1)
            correct += (pred == yb).sum().item()
            total += yb.size(0)

    return running_loss / total, correct / total


In [27]:
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

def get_device():
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def to_device(batch, device):
    x, y = batch
    return x.to(device, non_blocking=True), y.to(device, non_blocking=True)

In [1]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from torch import amp
import json

seed=2025
set_seed(seed)
device = get_device()
print("Device:", device)

DATA_ROOT = data_root 
BATCH = 32
EPOCHS = 500
LR = 3e-4
FREEZE_BACKBONE = False  # set True to train only the last layer first

counts = torch.tensor([670, 653, 328, 652], dtype=torch.float32)
beta = 0.999  # can be tuned in [0.99, 0.9999]

effective_num = 1.0 - beta ** counts
weights = (1.0 - beta) / effective_num
weights = weights / weights.sum() * len(counts)
print(weights)

#trainset, valset, testset, class_names = build_datasets(DATA_ROOT, img_size=224, val_fraction=0.15)
print("Classes:", class_names)

scaler = amp.GradScaler('cuda', enabled=(device.type=='cuda'))

train_loader = DataLoader(trainset,batch_size=BATCH,sampler=sampler,num_workers=WORKERS,pin_memory=True)                    # optional: keeps batch size stable)
val_loader = DataLoader(valset, batch_size=BATCH*2, shuffle=False,num_workers=WORKERS, pin_memory=True, drop_last=False)
test_loader = DataLoader(testset, batch_size=BATCH*2, shuffle=False,num_workers=WORKERS, pin_memory=True, drop_last=False)   

print("Train uses sampler:", train_loader.sampler.__class__.__name__)
print("Val uses sampler:  ", val_loader.sampler.__class__.__name__)
print("Test uses sampler: ", test_loader.sampler.__class__.__name__)

model = build_model(n_classes=len(class_names), freeze_backbone=FREEZE_BACKBONE)
#model = build_model(n_classes=len(class_names))

model.to(device)

#model = CNN512x3(num_classes=4, in_ch=3, dropout=0.5).to(device)
#criterion = nn.CrossEntropyLoss()  
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.05)
#criterion = FocalLoss(
#    gamma=1.5,    # 1.0 = CE; 1.5–2.0 is often best
#    reduction='mean'
#)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

history = {
    "train_loss": [],
    "val_loss":   [],
    "train_acc":  [],
    "val_acc":    [],
}


best_loss, best_acc, best_path, best_path_acc = 1000, 0.0, "best_ResNext_Loss_brain_tumor.pt", "best_ResNext_Acc_brain_tumor.pt"
stopper = EarlyStopper(patience=7, min_delta=1e-4)

for epoch in range(1, EPOCHS+1):
    train_loss, train_acc = epoch_run(model, train_loader, criterion, device, train=True, use_amp=True, optimizer=optimizer, scaler=scaler)
    val_loss,   val_acc   = epoch_run(model, val_loader,   criterion, device, train=False, use_amp=True)
    #scheduler.step(val_loss)
    scheduler.step()
    print(f"Epoch {epoch:02d}/{EPOCHS} | "
          f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
          f"val_loss={val_loss:.4f} acc={val_acc:.4f}")

    # log
    history["train_loss"].append(train_loss)
    history["val_loss"].append(val_loss)
    history["train_acc"].append(train_acc)
    history["val_acc"].append(val_acc)

    #Save model
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save({"model": model.state_dict(),
                    "classes": class_names}, best_path)
        
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({"model": model.state_dict(),
                    "classes": class_names}, best_path_acc)



    if stopper.step(val_acc):
        print("Early stopping triggered.")
        break


print(f"Best val loss: {best_loss:.4f}. Saved to: {best_path}")

with open("history_ResNext.json", "w") as f:
    json.dump(history, f)
# ---- after training: plots ----
epochs = range(1, len(history["train_loss"]) + 1)

plt.figure()
plt.plot(epochs, history["train_loss"], label="Train loss")
plt.plot(epochs, history["val_loss"],   label="Val loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.show()

# (optional) accuracy plot
plt.figure()
plt.plot(epochs, history["train_acc"], label="Train acc")
plt.plot(epochs, history["val_acc"],   label="Val acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.legend()
plt.grid(True)
plt.show()
# ----------------------------
# Evaluation on test (if available)
# ----------------------------
if test_loader is not None:
    ckpt = torch.load(best_path, map_location=device)
    model.load_state_dict(ckpt["model"])
    model.eval()
    all_preds, all_tgts = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = to_device((xb, yb), device)
            logits = model(xb)
            all_preds.append(logits.argmax(1).cpu())
            all_tgts.append(yb.cpu())


    y_pred = torch.cat(all_preds).numpy()
    y_true = torch.cat(all_tgts).numpy()
    print("\nTest classification report:")
    print(classification_report(y_true, y_pred, target_names= class_names, digits=4))
    print("\nConfusion matrix:")
    print(confusion_matrix(y_true, y_pred))

NameError: name 'set_seed' is not defined

In [11]:
from sklearn.metrics import classification_report, confusion_matrix

val_loader = DataLoader(valset, batch_size=BATCH*2, shuffle=False,num_workers=WORKERS, pin_memory=True, drop_last=False)
best_path = "best_ResNext_Loss_brain_tumor.pt"

device = get_device()
print("Device:", device)

if val_loader is not None:
    ckpt = torch.load(best_path, map_location=device)
    model.load_state_dict(ckpt["model"])
    model.to(device)
    model.eval()
    all_preds, all_tgts = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = to_device((xb, yb), device)
            logits = model(xb)
            all_preds.append(logits.argmax(1).cpu())
            all_tgts.append(yb.cpu())


    y_pred = torch.cat(all_preds).numpy()
    y_true = torch.cat(all_tgts).numpy()
    print("\nTest classification report:")
    print(classification_report(y_true, y_pred, target_names= class_names, digits=4))
    print("\nConfusion matrix:")
    print(confusion_matrix(y_true, y_pred))

Device: cuda


/tmp/ipykernel_13172/528204293.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(best_path, map_location=device)



Test classification report:
                  precision    recall  f1-score   support

    glioma_tumor     1.0000    0.9935    0.9967       154
meningioma_tumor     0.9941    1.0000    0.9970       168
        no_tumor     1.0000    1.0000    1.0000        74
 pituitary_tumor     1.0000    1.0000    1.0000       184

        accuracy                         0.9983       580
       macro avg     0.9985    0.9984    0.9984       580
    weighted avg     0.9983    0.9983    0.9983       580


Confusion matrix:
[[153   1   0   0]
 [  0 168   0   0]
 [  0   0  74   0]
 [  0   0   0 184]]
